In [ ]:
#@title # Create a Character Card v3 incl. World (SillyTavern) { display-mode: "form" }
#@markdown ⬅️❗Run this code first to initialize all parameters.

#@markdown Throughout this notebook, you will create your CharCard_v3 step by step:
#@markdown 1. Step: Creator Metadata
#@markdown 2. Step: System Prompt & Jailbreak (Optional)
#@markdown 3. Step: Character Personality (Essential)\
#@markdown    3.1. Step: Character Personality (Essential)
#@markdown 4. Step: Personality Depth (Optional)
#@markdown 5. Step: World & Complexity (Optional)\
#@markdown    5.1. Step: Keywords and Description

#@markdown How many tokens your character generates (gpt-3.5-turbo)

#@markdown 6. Step: Choose a image for your CharCard\
#@markdown    6.1. Step: Finalization\
#@markdown    6.2. Step: Save and Download your Char_Card_v3


import os
import sys

sys.stdout = open(os.devnull, 'w')

!pip show tiktoken || pip install tiktoken
!pip show panel || pip install panel
!pip show jupyter_bokeh || pip install jupyter_bokeh
!pip show timezonefinder || pip install timezonefinder


import json
import time
import zlib
import param
import base64
import struct
import tiktoken
import panel as pn
from PIL import Image
from io import BytesIO
from pathlib import Path
from datetime import datetime
from functools import partial
from zoneinfo import ZoneInfo
from PIL import Image, UnidentifiedImageError

try:
    from google.colab import files
except ImportError:
    files = None



eu = ZoneInfo("Europe/Berlin")

tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

pn.extension(
    theme='dark',
    raw_css=["textarea { resize: none; }"]
)

""" Serves for the modular definition and organization of properties. """
class CreatorMeta(param.Parameterized):
    creator = param.String(
        default="", doc="Character Card creator"
        )
    creator_notes = param.String(
        default="", doc="Notes from the creator"
        )
    character_version = param.String(
        default="1.0.0", doc="Version number of the Character Card"
        )
    tags = param.String(
        default="", doc="Example: Elf, Fantasy, Medieval"
        )

class PromptOverrides(param.Parameterized):
    system_prompt = param.String(
        default="", doc="Custom system prompt that replaces default instructions."
        )
    post_history_instructions = param.String(
        default="", doc="“Jailbreak” can allow filters to be bypassed."
        )

class Personality(param.Parameterized):
    char_name = param.String(
        default="", doc="Character name"
        )
    description = param.String(
        default="", doc="Character description"
        )
    personality = param.String(
        default="", doc="Character personality traits\nExample: shy + intelligent + friendly"
        )
    scenario = param.String(
        default="", doc="Character's scenario or backstory\nDo NOT use <START> here"
        )
    first_mes = param.String(
        default="", doc="Character's first message or greeting\nDo NOT use <START> here"
        )
    mes_example = param.String(
        default="", doc="Example message from the character\nStart each example message with <START>"
        )

class CharExtens_0(param.Parameterized):
    depth_prompt = param.String(
        default="", doc="Prompt to deepen the character's personality"
        )

class CharExtens_1(param.Parameterized):
    depth = param.Integer(4, bounds=(1, 6))
    talkativeness = param.Number(0.5, bounds=(0, None), softbounds=(0, 1))

class CharAlt(param.Parameterized):
    alternate_greetings = param.String(
        default="", doc="Alternative greetings from the character"
        )

class World(param.Parameterized):
    world = param.String(default=None, doc="Name of the world. Example: Eldoria")

class WorldKeys(param.Parameterized):
    keys = param.String(
        default="", doc="Keywords or categories for the world description\nExample: eldoria, forest, magical forest"
        )
    content = param.String(
        default="", doc="<START>\n{{user}}: What is Eldoria?\n{{char}}: Eldoria is an ancient, magical forest filled with mysterious creatures and glowing plants. It is considered the heart of nature magic and holds many hidden paths and mystical places only the bravest explore."
        )


def create_widgets(param_instance, max_items_per_row=2, exclude_counters=None, y=100, x=400, only_param=None):
    """ Dynamically creates widgets for input and display of parameter values."""
    if exclude_counters is None:
        exclude_counters = []

    widgets = []
    row = []

    for param_name, param_obj in param_instance.param.objects("existing").items():

        if only_param is not None and param_name not in only_param:
            continue

        if isinstance(param_obj, param.String) and param_name not in ['name']:
            label = pn.pane.Markdown(f"### {param_name.replace('_', ' ').capitalize()}")
            placeholder_text = param_obj.doc if param_obj.doc else f"Enter your {param_name} here..."
            text_area = pn.widgets.TextAreaInput(
                value=getattr(param_instance, param_name),
                placeholder=placeholder_text,
                height=y,
                width=x,
                max_length=10000
            )

            if param_name not in exclude_counters:
                char_counter = pn.pane.Markdown("**0 characters ~ 0 Tokens by Tiktoken**")

                def update_char_count(event, char_counter, param_name=param_name):
                    text = event.new
                    setattr(param_instance, param_name, text)
                    char_count = len(text)
                    token_count = len(tokenizer.encode(text))
                    char_counter.object = f"**{char_count} characters ~ {token_count} Tokens by Tiktoken**"

                text_area.param.watch(partial(update_char_count, char_counter=char_counter), 'value')

                row.append(pn.Column(label, text_area, char_counter))
            else:
                def update_value_no_counter(event, pn_name=param_name):
                    setattr(param_instance, pn_name, event.new)

                text_area.param.watch(partial(update_value_no_counter, pn_name=param_name), 'value')

                row.append(pn.Column(label, text_area))

            if len(row) == max_items_per_row:
                widgets.append(pn.Row(*row))
                row = []

    if row:
        widgets.append(pn.Row(*row))

    return widgets


def read_param_values(param_instance, only_params=None):
    """
    Reads all parameter values from the param_instance and returns them as a dictionary.
    If only_params is specified (list of strings), only those parameters will be read.
    """
    values_dict = {}

    for param_name, param_obj in param_instance.param.objects("existing").items():
        if isinstance(param_obj, param.String) and param_name not in ['name']:
            if only_params is not None and param_name not in only_params:
                continue

        values_dict[param_name] = getattr(param_instance, param_name)

    return values_dict

"""
Creates a Panel layout with a confirmation button and an output display area.
"""
def create_confirm_button():

    button = pn.widgets.Button(name='Confirm Input')
    output = pn.pane.Markdown("")

    def animate_output():
        confirm_time = datetime.now(eu).strftime("%H:%M:%S")
        for dots in [".", "..", "..."]:
            output.object = dots
            time.sleep(0.2)
            pn.io.push_notebook()  # Notebook manuell aktualisieren

        output.object = f"### <span style='color:green;'>**Input confirmed [{confirm_time}]**</span>"

    def on_button(event):
        pn.state.execute(animate_output)  # Sicherstellen, dass Animation nicht blockiert

    button.on_click(on_button)
    return pn.Column(button, output)

"""
Dynamically creates a dictionary of button panels, where
each key is an index and each value is a confirmation button widget.
"""
button_panels = {i: create_confirm_button() for i in range(1, 10)}


# Instance of the classes
creator_meta = CreatorMeta()
jailbreak = PromptOverrides()
personality = Personality()
char_extens = CharExtens_0()
char_extens_1 = CharExtens_1()
char_alt = CharAlt()
world_name = World()


# Widgets for all parameters
widgets_0 = create_widgets(creator_meta, y=35, exclude_counters=['creator', 'creator_notes', 'character_version', 'tags'])
widgets_1 = create_widgets(jailbreak, x=500)
widgets_2 = create_widgets(personality, y=35, only_param=['char_name'])
widgets_3 = create_widgets(personality, x=600, y=300, only_param=['description', 'personality'])
widgets_3_1 = create_widgets(personality, x=600, y=300, only_param=['scenario', 'first_mes', 'mes_example'])
widgets_4 = create_widgets(char_extens, x=600, y=300, only_param=['depth_prompt'])
widgets_5 = create_widgets(char_alt, x=600, y=300)
widgets_6 = create_widgets(world_name, y=35, exclude_counters=['world'])

sys.stdout = sys.__stdout__
pn.pane.Markdown("The parameters are now initialized. You can now proceed to step 1.")


In [ ]:
#@title 1. Step: Creator Metadata { display-mode: "form" }
#@markdown In this section, you can specify information about your CharCard.\
#@markdown Entries in this area do not generate tokens.
pn.Column(
    pn.pane.Markdown("## Meta Daten deiner CharCard"),
    *widgets_0,
    button_panels[1]
)


In [ ]:
#@title 2. Step: System Prompt & Jailbreak (Optional) { display-mode: "form" }
#@markdown In this section, you can optionally enter a "Jailbreak" (not recommended).


pn.Column(
    pn.pane.Markdown("## CharCard"),
    *widgets_1,
    button_panels[2]
)

In [ ]:
#@title 3. Step: Character Personality (Essential!) { display-mode: "form" }
#@markdown In this step, you define your character's personality profile.

pn.Column(
    pn.pane.Markdown(
        "# Persönlichkeit des Charakters\n"
        ),
    *widgets_2,
    *widgets_3,
    button_panels[3],
    height_policy="fit"
    )

In [ ]:
#@title 3.1. Step: Character Personality Context (Essential!) { display-mode: "form" }
#@markdown When describing actions, use "*non-verbal actions*".\
#@markdown Some of these fields require a specific format.\
#@markdown Formatting guidelines:
#@markdown + Use `{{char}}` instead of the character name.
#@markdown + Use `{{user}}` instead of the username.

#@markdown Example:
#@markdown ```
#@markdown <START>
#@markdown {{user}}: Hey Aqua, how’s your adventure going?
#@markdown {{char}}: *laughs loudly* Oh, my adventures? They mostly end in chaos, but hey, that’s part of the fun, right?
#@markdown ```

pn.Column(
    pn.pane.Markdown(
        "# Charakter Kontext\n"
        ),
    *widgets_3_1,
    button_panels[4],
    )


In [ ]:
#@title 4. Step: Personality Depth (Optional) { display-mode: "form" }
#@markdown Adding depth to your character's personality makes them maintain their personality more consistently.\
#@markdown * Less freedom in character development.\
#@markdown * You can add an alternative greeting message.
pn.Column(
    pn.pane.Markdown("## Ersteller\n"),
    pn.Row(
        *widgets_4,
        *widgets_5
    ),
    pn.Param(char_extens_1.param, show_name=False),
    button_panels[5]
)

In [ ]:
#@title 5. Step: World & Complexity (Optional) { display-mode: "form" }
#@markdown Here, you specify the name of the world in which your character lives.\
#@markdown * Decide how complex your world should be.\
#@markdown This is only recommended for models that can handle a large context.\
#@markdown (My recommendation: Claude 3.5 Haiku)
class ValueInfo(param.Parameterized):
    value_infos = param.Integer(4, bounds=(1, 12))

value_info = ValueInfo()

pn.Column(
    pn.pane.Markdown(
        "## Number of World Details\n"
        "Warning! You will need a lot of information here.\n"
        "If you want your world to be complex, I recommend using 4 world details."
        ),
    pn.Param(value_info.param, show_name=False),
    *widgets_6,
    button_panels[6]
    )

In [ ]:
#@title 5.1. Step: Keywords and Description { display-mode: "form" }
#@markdown Warning: *Only execute this step if you have completed Step 5!*

value = value_info.value_infos
name = world_name.world
world_keys_list = []

if name is not None and not len(name) == 0:
    def world_value(value, name):

        world_keys = WorldKeys()
        world_keys_list.append(world_keys)

        widgets_7 = create_widgets(world_keys, y = 60, only_param=['keys'])
        widgets_8 = create_widgets(world_keys, x=600, y=300, only_param=['content'])

        return pn.Column(
            pn.pane.Markdown(f"## {value+1}. Description of {name}"),
            pn.Column(
                *widgets_7,
                *widgets_8
            )
        )

    panels = []
    for i in range(value):
        panels.append(world_value(i, name))


else:
    panels = pn.pane.Markdown('You did not provide a world name!')

pn.Column(*panels, button_panels[7])


In [ ]:
#@title #####Debug (You can ignore this section) { display-mode: "form" }
button = pn.widgets.Button(name='Read Parameters')

def on_button_click(event):
    button.disabled = True
    all = [
        read_param_values(creator_meta),
        read_param_values(jailbreak),
        read_param_values(personality),
        read_param_values(char_extens),
        read_param_values(char_extens_1),
        read_param_values(char_alt),
        read_param_values(world_name),
    ]
    if 'world_keys_list' in globals():
        for world_keys in world_keys_list:
            all.append(read_param_values(world_keys))

    print(all)

button.on_click(on_button_click)

pn.Column(button)

In [ ]:
#@title Learn how many tokens your character generates (gpt-3.5-turbo) { display-mode: "form" }
#@markdown 2048 tokens are recommended, but some models (like Claude) can handle an input of up to 100k tokens.\
#@markdown - From experience: Less is more.\
#@markdown (Remember that memory also requires tokens.)\
#@markdown Rough example: Total 8192 tokens - Personality 2048 tokens = Memory 6144 tokens.\
#@markdown The total number of tokens depends on your model and settings.

# Function to calculate the total number of tokens for a parameter
def total_token_count(param_instance, encoding):
    total = 0
    for param_name, param_obj in param_instance.param.objects("existing").items():
        if isinstance(param_obj, param.String) and param_name not in ['name']:
            wert = getattr(param_instance, param_name)
            if wert:
                tokens = len(encoding.encode(wert))
                total += tokens
    return total

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

gesamt_tokens = (
    total_token_count(personality, encoding) +
    total_token_count(jailbreak, encoding) +
    total_token_count(char_extens, encoding) +
    total_token_count(char_extens_1, encoding) +
    total_token_count(char_alt, encoding)
)
if 'world_keys_list' in globals():
    for world_keys in world_keys_list:
      gesamt_tokens += (total_token_count(world_keys, encoding))



if gesamt_tokens >= 2048:
    color = "yellow"
else:
    color = "green"



markdown_text = f"# Gesamt: <span style='color:{color};'>Tokens {gesamt_tokens}</span>"

pn.pane.Markdown(markdown_text).servable()

In [ ]:
#@title 6. Step: Choose a image for your CharCard { display-mode: "form" }
#@markdown In this step, you can upload an image for your CharCard.\
#@markdown The allowed file formats are: `.png`, `.jpg`, `.jpeg`, `.webp`

# Create the upload path if it does not exist
upload_path = 'image_upload'
os.makedirs(upload_path, exist_ok=True)

# Filename for the uploaded image
PREVIEW_FILENAME = "upload_preview.png"

# "Choose File"-Button
upload = pn.widgets.FileInput(
    accept='image/*',
    multiple=False,
    name='Bilder hochladen'
)

# "Image delete"-Button 
delete_button = pn.widgets.Button(name="Image delete", button_type="danger")

# GridBox for image preview
vorschau = pn.GridBox(
    name='Vorschau',
    ncols=3,
    sizing_mode='stretch_width'
)

#=================================
# Function for handling the upload
#=================================
def handle_upload(event):
    # Reset the preview and delete existing files
    vorschau[:] = []
    # Delete existing files
    delete_existing_files()
    # Check if a file was uploaded
    if not upload.value:
        return
    
    # Get the file bytes
    file_bytes = upload.value
    try:
        # Load the image from the file bytes
        image = Image.open(BytesIO(file_bytes))
        image_format = image.format.lower() if image.format else 'png'
        filename = f"{PREVIEW_FILENAME.split('.')[0]}.{image_format}"

        # Save the image to the upload path
        dateipfad = os.path.join(upload_path, filename)
        image.save(dateipfad)
        print(f'Bild gespeichert: {dateipfad}')

        # Prepare the image for display
        buffered = BytesIO()
        image.save(buffered, format=image_format)
        encoded = base64.b64encode(buffered.getvalue()).decode('utf-8')
        data_url = f"data:image/{image_format};base64,{encoded}"

        # Display the image in the preview
        image_pane = pn.pane.HTML(f"<img src='{data_url}' width='300' />")
        vorschau.append(pn.Column(image_pane, pn.pane.Str(f"Dateiname: {filename}")))

        # Hide the upload button and show the delete button
        upload.visible = False
        delete_button.visible = True

    # Handle errors
    except UnidentifiedImageError:
        error_message = "<div style='color:red;'><strong>Fehler:</strong> Datei ist kein gültiges Bild.</div>"
        vorschau.append(pn.pane.HTML(error_message))
        print(f"Fehler: Datei ist kein gültiges Bild.")

    except Exception as e:
        error_message = f"<div style='color:red;'><strong>Fehler beim Hochladen:</strong> {e}</div>"
        vorschau.append(pn.pane.HTML(error_message))
        print(f"Fehler beim Hochladen: {e}")

#=================================
# Function to delete existing files
#=================================
def delete_existing_files():
    for file in os.listdir(upload_path):
        file_path = os.path.join(upload_path, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f'Datei gelöscht: {file_path}')

#=================================
# Function to handle the delete button
#=================================
def handle_delete(event):
    delete_existing_files()  # Lösche die aktuelle Datei
    vorschau[:] = []  # Vorschau zurücksetzen
    upload.visible = True  # "Choose File"-Button wieder einblenden
    delete_button.visible = False  # "Löschen"-Button ausblenden

# Event-Handling registrieren
upload.param.watch(handle_upload, 'value')
delete_button.on_click(handle_delete)

# Standardmäßig "Datei löschen"-Button ausblenden
delete_button.visible = False

# Layout der Anwendung
pn.Column(
    pn.pane.HTML("<h2>Bild-Upload Anwendung</h2>"),
    upload,
    delete_button,
    vorschau
)

In [ ]:
#@title 6.1. Step: Finalization { display-mode: "form" }
#@markdown Your entries will now be converted into a `.json` format.\
#@markdown Start the code and save it in Step 6.1.

all_para = [
    read_param_values(creator_meta),
    read_param_values(jailbreak),
    read_param_values(personality),
    read_param_values(char_extens),
    read_param_values(char_extens_1),
    read_param_values(char_alt),
    read_param_values(world_name)
    ]

world_para = []
if 'world_keys_list' in globals():
    for world_keys in world_keys_list:
        world_para.append(read_param_values(world_keys))

day = datetime.now(eu).strftime("%Y-%m-%d")
time = datetime.now(eu).strftime("%Hh %Mm %Ss %Z")
# Initialisieren der JSON-Struktur
json_data = {
    "name": "",
    "description": "",
    "personality": "",
    "first_mes": "",
    "avatar": "none",
    "chat": "",
    "mes_example": "",
    "scenario": "",
    "create_date": f"{day} @{time}",
    "talkativeness": "",
    "fav": False,
    "creatorcomment": "",
    "spec": "chara_card_v3",
    "spec_version": "3.0",
    "data": {
        "name": "",
        "description": "",
        "personality": "",
        "scenario": "",
        "first_mes": "",
        "mes_example": "",
        "creator_notes": "",
        "system_prompt": "",
        "post_history_instructions": "",
        "tags": [],
        "creator": "",
        "character_version": "",
        "alternate_greetings": [],
        "extensions": {
            "talkativeness": "0.5",
            "fav": False,
            "world": "",
            "depth_prompt": {
                "prompt": "",
                "depth": 4,
                "role": "system"
            }
        },
        "character_book": {
            "entries": [],
            "name": ""
        },
        "group_only_greetings": []
    },
    "tags": [],
    "metadata": {
        "tool": {
            "developer": "Sakushi-Dev",
            "version": "1.0.0",
            "url": "https://github.com/Sakushi-Dev/CharCard-Creator"

        }
    }
}


def assign_parameters(params, json_struct):
    for param in params:

        name = param.get('name')

        if name.startswith('CreatorMeta'):

            json_struct['creatorcomment'] = param.get('creator_notes', '')
            json_struct['data']['creator'] = param.get('creator', '')
            json_struct['data']['creator_notes'] = param.get('creator_notes', '')
            json_struct['data']['character_version'] = param.get('character_version', '')

            tags = param.get('tags', '')
            if isinstance(tags, list):
                json_struct['data']['tags'] = tags
                json_struct['tags'] = tags
            elif isinstance(tags, str):
                json_struct['data']['tags'] = [tags]
                json_struct['tags'] = tags

        elif name.startswith('PromptOverrides'):
            json_struct['data']['system_prompt'] = param.get('system_prompt', '')
            json_struct['data']['post_history_instructions'] = param.get('post_history_instructions', '')

        elif name.startswith('Personality'):
            json_struct['name'] = param.get('char_name', '')
            json_struct['description'] = param.get('description', '')
            json_struct['personality'] = param.get('personality', '')
            json_struct['scenario'] = param.get('scenario', '')
            json_struct['first_mes'] = param.get('first_mes', '')
            json_struct['mes_example'] = param.get('mes_example', '')

            json_struct['data']['name'] = param.get('char_name', '')
            json_struct['data']['description'] = param.get('description', '')
            json_struct['data']['personality'] = param.get('personality', '')
            json_struct['data']['scenario'] = param.get('scenario', '')
            json_struct['data']['first_mes'] = param.get('first_mes', '')
            json_struct['data']['mes_example'] = param.get('mes_example', '')

        elif name.startswith('CharExtens'):
            for key, value in param.items():
                if key == 'name':
                    continue
                if key == 'depth_prompt' and isinstance(value, str):
                    json_struct['data']['extensions']['depth_prompt']['prompt'] = value
                elif key == 'depth':
                    json_struct['data']['extensions']['depth'] = value
                elif key == 'talkativeness':
                    json_struct['talkativeness'] = f"{value}"
                    json_struct['data']['extensions']['talkativeness'] = f"{value}"

        elif name.startswith('World'):
            for key, value in param.items():
                if key == 'name':
                    continue
                elif key == 'world':
                    json_struct['data']['extensions']['world'] = value
                    json_struct['data']['character_book']['name'] = value

        elif name.startswith('CharAlt'):
            alternate_greetings = param.get('alternate_greetings', [])
            if isinstance(alternate_greetings, str):
                json_struct['data']['alternate_greetings'].append(alternate_greetings)
            elif isinstance(alternate_greetings, list):
                json_struct['data']['alternate_greetings'].extend(alternate_greetings)

def add_world_entries(world_keys, json_struct):
    id = 0

    for world_key in world_para:
        key = world_key.get('keys')
        content = world_key.get('content')

        entry = {
            "id": id,
            "keys": [key],
            "secondary_keys": [],
            "comment": "",
            "content": content,
            "constant": False,
            "selective": True,
            "insertion_order": 100,
            "enabled": True,
            "position": "before_char",
            "use_regex": True,
            "extensions": {
                "position": 0,
                "exclude_recursion": False,
                "display_index": 0,
                "probability": 0,
                "useProbability": True,
                "depth": 4,
                "selectiveLogic": 0,
                "group": "",
                "group_override": False,
                "group_weight": 100,
                "prevent_recursion": False,
                "delay_until_recursion": False,
                "scan_depth": None,
                "match_whole_words": None,
                "use_group_scoring": False,
                "case_sensitive": None,
                "automation_id": "",
                "role": 0,
                "vectorized": False,
                "sticky": 0,
                "cooldown": 0,
                "delay": 0
            }
        }
        json_struct['data']['character_book']['entries'].append(entry)

        id+=1

assign_parameters(all_para, json_data)

if 'world_keys_list' in globals():
    add_world_entries(world_para, json_data)

pn.pane.Markdown("You can now save and download your CharCard.")


In [ ]:
#@title #####Debug (You can ignore this section) { display-mode: "form" }
pn.pane.Markdown(json.dumps(json_data, indent=4, ensure_ascii=False))

In [ ]:
#@title 6.2. Step: Save and Download { display-mode: "form" }

# ========================
# User-Defined Exceptions
# ========================
class PngEncodeError(Exception):
    pass

class PngDecodeError(Exception):
    pass

class PngMissingCharacterError(Exception):
    pass

class PngInvalidCharacterError(Exception):
    pass

# ========================
# PNG-Processing Functions
# ========================
class Png:

    #=========================
    # static method to read chunks
    #=========================
    @staticmethod
    def read_chunks(data):
        """Reads all PNG chunks from a PNG file."""
        if data[:8] != b'\x89PNG\r\n\x1a\n':
            raise PngEncodeError("Invalid PNG header")

        chunks = []
        idx = 8  

        while idx < len(data):
            if idx + 8 > len(data):
                raise PngDecodeError("Incomplete PNG chunk header")

            length = struct.unpack('>I', data[idx:idx + 4])[0]
            idx += 4

            chunk_type = data[idx:idx + 4].decode('ascii')
            idx += 4

            chunk_data = data[idx:idx + length]
            idx += length

            if idx + 4 > len(data):
                raise PngDecodeError("Incomplete PNG CRC")
            crc = struct.unpack('>I', data[idx:idx + 4])[0]
            idx += 4

            chunks.append({
                'type': chunk_type,
                'data': chunk_data,
                'crc': crc,
            })

        if not chunks or chunks[-1]['type'] != 'IEND':
            raise PngDecodeError("Missing IEND chunk")

        return chunks
    
    #=========================
    # static method to write chunks
    #=========================

    @staticmethod
    def write_chunks(chunks):
        """Writes a list of PNG chunks to a PNG file."""
        png_signature = b'\x89PNG\r\n\x1a\n'
        output = bytearray(png_signature)

        for chunk in chunks:
            length = len(chunk['data'])
            output += struct.pack('>I', length)
            output += chunk['type'].encode('ascii')
            output += chunk['data']
            crc_data = chunk['type'].encode('ascii') + chunk['data']
            crc = struct.pack('>I', (zlib.crc32(crc_data) & 0xffffffff))
            output += crc 

        return output
    
    #=========================
    # static method to resize height
    #=========================

    @staticmethod
    def resize_height(img, target_height):
        """
        skale the image to the target height while maintaining the aspect ratio
        and return the resize, if the width is less than 512, adjust the width to 512
        """
        width_percent = (target_height / float(img.size[1]))
        new_width = int((float(img.size[0]) * float(width_percent)))
        resized_img = img.resize((new_width, target_height), Image.Resampling.LANCZOS)
        
        print(f"Skale the Image to: {resized_img.size}")
        
        # Adjust the width to 512 if it is less than 512
        if resized_img.size[0] < 512:
            print("Breite kleiner als 512. Passe Bildgröße an...")
            aspect_ratio = resized_img.size[0] / resized_img.size[1]
            new_width = 512
            new_height = int(new_width / aspect_ratio)
            resized_img = resized_img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            print(f"Bild angepasst auf: {resized_img.size}")
        
        return resized_img
    
    #=========================
    # static method to crop center
    #=========================

    @staticmethod
    def crop_center(img, target_width, target_height):
        """Crops the image to the target width and height, centered."""
        img_width, img_height = img.size
        left = max((img_width - target_width) // 2, 0)
        top = max((img_height - target_height) // 2, 0)
        right = left + target_width
        bottom = top + target_height
        cropped_img = img.crop((left, top, right, bottom))
        print(f"Image crops to: {cropped_img.size}")
        return cropped_img
    
    #===========================================
    # static method to resize and crop png bytes
    #===========================================	

    @staticmethod
    def resize_and_crop_png_bytes(image_bytes, target_width, target_height):
        """Scale and crop the image to the target width and height."""
        try:
            img = Image.open(BytesIO(image_bytes))
            print("Image data opened successfully.")
        except Exception as e:
            raise PngDecodeError(f"Cannot open image data: {e}")

        img = img.convert("RGBA")  # ensure image has an alpha channel

        # Scale the height to target_height pixels
        resized_img = Png.resize_height(img, target_height)

        # Scale the width to target_width pixels
        cropped_img = Png.crop_center(resized_img, target_width, target_height)

        # Image conversion to PNG
        with BytesIO() as buffer:
            cropped_img.save(buffer, format='PNG')
            resized_png_data = buffer.getvalue()
            print(f"Resized and cropped PNG-Datengröße: {len(resized_png_data)} Bytes")
            return resized_png_data
        
    #=========================
    # static method to convert to png
    #=========================

    @staticmethod
    def convert_to_png(image_bytes, original_format):
        """Converts an image in a given format to PNG."""
        try:
            img = Image.open(BytesIO(image_bytes))
            img = img.convert("RGBA")  # ensure image has an alpha channel
            with BytesIO() as buffer:
                img.save(buffer, format='PNG')
                png_data = buffer.getvalue()
                print(f"Konvertierung von {original_format.upper()} zu PNG erfolgreich. PNG-Datengröße: {len(png_data)} Bytes")
                return png_data
        except Exception as e:
            raise PngEncodeError(f"Failed to open image for conversion: {e}")
        
    #=========================
    # static method to embed text chunk
    #=========================

    @staticmethod
    def embed_text_chunk(file_path, json_path, output_path):
        """Accepts a PNG file, a JSON file, and an output path. Embeds the JSON data in the PNG file."""
        target_width = 512
        target_height = 768

        try:
            with open(file_path, 'rb') as f:
                img_data = f.read()
                img = Image.open(BytesIO(img_data))
                original_format = img.format.lower()
                print(f"Bilddatei '{file_path}' geöffnet. Format: {original_format.upper()}")
        except Exception as e:
            raise PngEncodeError(f"Failed to open image file '{file_path}': {e}")

        # Check if the image is already in PNG format
        if original_format != 'png':
            print(f"Konvertiere {original_format.upper()} zu PNG.")
            try:
                png_data = Png.convert_to_png(img_data, original_format)
            except Exception as e:
                raise PngEncodeError(f"Failed to convert {original_format} to PNG: {e}")
        else:
            png_data = img_data
            print("Bild ist bereits im PNG-Format.")

        # Scale and crop the image to the target dimensions
        try:
            resized_png_data = Png.resize_and_crop_png_bytes(png_data, target_width, target_height)
        except Exception as e:
            raise PngDecodeError(f"Failed to resize and crop image: {e}")

        # Read the JSON data
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                json_data = f.read()
                print(f"JSON-Daten aus '{json_path}' erfolgreich gelesen.")
        except Exception as e:
            raise PngEncodeError(f"Failed to read JSON file '{json_path}': {e}")

        # Base64-code the JSON data
        encoded_text = f'chara\0{base64.b64encode(json_data.encode("utf-8")).decode("utf-8")}'.encode('utf-8')
        print(f"JSON-Daten successfully base64-encoded.")

        # PNG-Chunks read
        try:
            chunks = Png.read_chunks(resized_png_data)
            print("PNG-Chunks successfully read.")
        except Exception as e:
            raise PngDecodeError(f"Failed to read PNG chunks: {e}")

        # Create a new tEXt chunk with the JSON data
        text_chunk = {
            'type': 'tEXt',
            'data': encoded_text,
            'crc': 0
        }

        # Add the new tEXt chunk before the IEND chunk
        chunks.insert(-1, text_chunk)
        print("tEXt-Chunk added successfully.")

        # Write the new PNG chunks with the embedded JSON data
        try:
            new_png_data = Png.write_chunks(chunks)
            print("New PNG chunks written successfully.")
        except Exception as e:
            raise PngEncodeError(f"Failed to write PNG chunks: {e}")

        # Save the new PNG file
        try:
            with open(output_path, 'wb') as f:
                f.write(new_png_data)
                print(f"New PNG file saved to '{output_path}'.")
        except Exception as e:
            raise PngEncodeError(f"Failed to write output PNG file '{output_path}': {e}")

        print(f"JJSON-Data successfully embedded in PNG file '{output_path}'.")

    #=========================
    # static method to parse
    #=========================

    @staticmethod
    def parse(file_path):
        """Extracts the JSON data from a PNG file."""
        try:
            with open(file_path, 'rb') as f:
                data = f.read()
            print(f"PNG-Data from '{file_path}' successfully read.")
        except Exception as e:
            raise PngDecodeError(f"Failed to open PNG file '{file_path}': {e}")

        chunks = Png.read_chunks(data)

        # Search chunks
        text_chunks = [chunk for chunk in chunks if chunk['type'] == 'tEXt']
        if not text_chunks:
            raise PngMissingCharacterError("No tEXt chunks found")

        # Search for 'chara' chunk
        for chunk in text_chunks:
            try:
                decoded_text = chunk['data'].decode('utf-8')
                print("tEXt-Chunk successfully decoded.")
            except UnicodeDecodeError:
                continue

            if decoded_text.startswith('chara\0'):
                chara_data = decoded_text[6:]  # remove 'chara\0' prefix
                try:
                    json_data = base64.b64decode(chara_data).decode('utf-8')
                    print("Chara-Data successfully base64-decoded.")
                    return json_data
                except Exception as e:
                    raise PngInvalidCharacterError("Failed to decode 'chara' data") from e

        raise PngMissingCharacterError("No 'chara' tEXt field found")
    
# ========================
# Helper Functions
# ========================

def search_image(ordner, endungen=['png', 'jpg', 'jpeg', 'webp']):
    """
    Browse the specified folder and search for image files with the specified extensions.
    """
    pfad = Path(ordner)
    
    if not pfad.exists():
        print(f"Der angegebene Ordner '{ordner}' existiert nicht.")
        return []
    
    if not pfad.is_dir():
        print(f"Der Pfad '{ordner}' ist kein Verzeichnis.")
        return []
    
    gefundene_dateien = []
    for datei in pfad.rglob('*'):
        if datei.is_file() and datei.suffix.lower().lstrip('.') in endungen:
            gefundene_dateien.append(datei)
    
    return gefundene_dateien


if __name__ == "__main__":
    try:
        base_name = personality.char_name if personality.char_name else "CharCard"

        json_filename = f"{base_name}.json"
        directory = Path('JSON')
        json_path = directory / json_filename
        directory.mkdir(parents=True, exist_ok=True)

        def get_unique_filename(filepath):
                counter = 1
                new_filepath = filepath
                while new_filepath.exists():
                    new_filename = f"{filepath.stem}_{counter}{filepath.suffix}"
                    new_filepath = filepath.parent / new_filename
                    counter += 1
                return new_filepath
        
        unique_filename = get_unique_filename(json_path)
        with unique_filename.open('w', encoding='utf-8') as file:
            json.dump(json_data, file, indent=4, ensure_ascii=False)
        
        path_to_file = r'./image_upload'  # filepath to the folder containing images
        image = search_image(path_to_file)
        # embed text chunk
        if image:
            for bild in image:
                print(f"Verarbeite: {bild}")
                input_image = bild  
                input_json = unique_filename
                output_path = Path('CharCard')
                output_path.mkdir(parents=True, exist_ok=True)
                output_png = output_path / f"{base_name}.png"  

                output_path.mkdir(parents=True, exist_ok=True)

                unique_filename = get_unique_filename(output_png)
                Png.embed_text_chunk(input_image, input_json, unique_filename)
            
            if files:
                files.download(unique_filename)
                message = f"Your CharCard has been saved and downloaded as `{unique_filename.name}`."
                pass
            else:
                message = (
                    f"The CharCard has been saved as `{unique_filename.name}`.\n"
                    "Unfortunately, the download is not supported in this environment."
                )
                pass

        else:
            print(f"Keine Bilder mit den Endungen {['png', 'jpg', 'jpeg']} in '{path_to_file}' gefunden.")
    except Exception as e:
        print("Fehler:", e)


    


pn.Row(pn.pane.Markdown(message))  